In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import re
import math

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_json("rawdatajson/products.json", lines=True)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 1347: ordinal not in range(128)

In [ ]:
df.head()

In [ ]:
list(df.columns)

In [ ]:
df.categories.value_counts()

In [ ]:
df.shape 

In [ ]:
df[1500:1520]

In [ ]:
allergens = [x for x in list(df.allergens_tags.values) if type(x) == list and len(x)>0]


In [ ]:
allergens

In [ ]:
### what are these allergens?
### how many allergens are there?
len(allergens)

In [ ]:
####
## prunus-amygdalus is almond, a tree nut
## aluminum is aluminum
## cinnamal is the naturally occuring chemical in cinnamon that gives it its flavor, banned for cosmetic use in EU
## citral is a chemical commonly found in lemon scented plants that is known to be a skin irritant
## macadamia is macademia nut, a tree nut
## arachidyl is derived from peanuts
## d-limonene is a constitutent of citurs and is known to cause skin irritation in some individuals
## linalool is a naturally occuring scent in lavendar, mint, orange blossoms, cardamom and others.  It causes skin allergies in about 5% of the Swedish population
## geraniol is naturally occuring and found in many essential oils that smell like roses.  It is banned in EU as a skin irritant
## alpha-isomethyl-ionone is an ingredient in bubble bath and aftershave that can cause skin irritation
## coumarin gives many plants their sweet smell and is a known human irritant.  Banned in EU
## citronellol is found in roses and geraniums.  It is restricted in perfume

### Sources https://www.ewg.org/skindeep/ , http://en.wikipedia.org

In [ ]:
main_allergens = [x for x in list(df.allergens.values) if type(x) == str and len(x) >0]

In [ ]:
type(df.allergens[75])
#type(df.allergens.values[20])

In [ ]:
main_allergens

In [ ]:
len(main_allergens)

In [ ]:
cats = [x for x in df['categories_hierarchy'] if type(x) == list  and len(x) >0]

In [ ]:
type(df['categories_hierarchy'][0])

In [ ]:
len(cats)

In [ ]:
### half of all cosmetics have category hierarchies

In [ ]:
cats_general = [x for x in df['categories'] if type(x) == str and len(x)>0]

In [ ]:
len(cats_general)

In [ ]:
cats_general

In [ ]:
cat_tags = [x for x in df['categories_tags'] if type(x) == list  and len(x) >0]

In [ ]:
cat_tags

In [ ]:
len(cat_tags)

In [ ]:
for x in df.columns:
    print (x, type(df[x][0]))


In [ ]:
### drop rows that are not relevant.  Define relevant as if categories_tags is filled
idx = []
for i, val in enumerate(df['categories_tags']):
    #print i, val, type(val)
    if (type(val) != list) or (type(val) == list and len(val)  == 0 ):
        idx.append(i)
idx[:7]

In [ ]:
df1 = df.drop(df.index[idx])
df1.head(10)

In [ ]:
df1.shape

In [ ]:
# re-index
df2 = df1.reset_index(drop=True)
df2.head(10)

In [ ]:
### check for completeness
cat_completeness = []
types = set()

for x in df2.columns:
    t = type(df2[x][1])
    types.add(t)
    df2[x] = df2[x].apply(lambda y: np.nan if (type(y)==list or type(y)==str) and len(y)==0 else y)
    df2[x] = df2[x].apply(lambda y: np.nan if type(y)==dict and (not bool(y)) else y)
    ln = df2[x].notnull().sum()
    threshold = ['Over Threshold' if ln > 2619*0.95 else 'Under Threshold'][0]
    cat_completeness.append({'cat_name': x, 
                            'cat_type': t,
                            'cat_valid_count': ln,
                            'cat_completeness': threshold})

In [ ]:
cat_completeness[:20]

In [ ]:
### how many were not complete?
cols2 = [x['cat_name'] for x in cat_completeness if x['cat_completeness'] == 'Over Threshold']
len(cols2)

In [ ]:
### create a df that only contains the completed columns
df3 = df2[cols2]

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
len(df3)

In [ ]:
df4 = df3.dropna()
df4.shape
df4.head(10)

In [ ]:
df4.shape

In [ ]:
writer = pd.ExcelWriter('cleaned_data.xlsx')
df4.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
df4.to_json('cleaned_data1.json', orient='records')

In [ ]:
### check to make sure the data saved as intended
cleaned_data = pd.read_json('cleaned_data1.json')

In [ ]:
cleaned_data.head()

In [ ]:
cleaned_data.shape